## 1. Web Crawling for Stock Related News

## 2. Scrolling Down Page to Get More New

## 3. Text Mining

In [ ]:
#import the library
import requests
from bs4 import BeautifulSoup

In [ ]:
#change artical content format for text mining
def artical_format(artical_content):
    artical_texts = []
    for i in artical_content:
        if len(i)>1:
            artical_texts.append([i[0]]+[''.join(i[1:])])
    return artical_texts

In [ ]:
#return text analysis results
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk import sent_tokenize
import pandas as pd
def vader_comparison(texts):
    headers = ['Name','pos','neg','neu','compound']
    result=pd.DataFrame(columns=headers)
    print("Name\t",'  pos\t','neg\t','neu\t','compound')
    analyzer = SentimentIntensityAnalyzer()
    for i in range(len(texts)):
        name = texts[i][0]
        sentences = sent_tokenize(texts[i][1])
        pos=compound=neu=neg=0
        for sentence in sentences:
            vs = analyzer.polarity_scores(sentence)
            pos+=vs['pos']/(len(sentences))
            neu+=vs['neu']/(len(sentences))
            neg+=vs['neg']/(len(sentences))
            compound+=vs['compound']/(len(sentences))
        result=result.append(pd.DataFrame([[name,pos,neg,neu,compound]],columns=headers))
        print('%-10s'%name,'%1.2f\t'%pos,'%1.2f\t'%neg,'%1.2f\t'%neu,'%1.2f\t'%compound)
    result=result.sort_values(by='Name')
    result.index=range(len(result))
    return result

In [ ]:
stock_name = 'AAPL'
all_links=get_article_link(stock_name)
artical_content=get_article_content(all_links)
artical_texts=artical_format(artical_content)
news_result=vader_comparison(artical_texts)

In [ ]:
from pandas_datareader import data as web
from datetime import datetime
import math
def stock_analysis(start, end, stock_name):
    print(start,end)
    stock_df = web.DataReader(stock_name, 'yahoo', start, end)
    stock_df['logReturn']=pd.Series([math.log(i) for i in stock_df['Close']/stock_df['Open']],
                                 index=stock_df.index)
    stock_df.describe() #Get summary statistics
    return stock_df

In [ ]:
start=datetime(2000, 1, 1)
end=datetime.today()
stock_data=stock_analysis(start, end, stock_name)

In [ ]:
from datetime import datetime
def news_stock_analysis(news_result, stock_data):
    headers = ['Name','pos','neg','neu','compound']
    news_result=news_result.groupby(['Name'])[headers[1:]].mean()
    news_result['Date']=[str(datetime.strptime(i,'%B %d, %Y'))[:10] for i in news_result.index]
    news_result.index=range(len(news_result))
    
    stock_data['Date']=[str(i)[:10] for i in stock_data.index]
    stock_data.index=range(len(stock_data))
    
    compound_data=news_result.merge(stock_data, on='Date', how='left')
    return compound_data

## 4.  Acquiring Stock Data

## 5. Data Visualization